In [2]:
import betfairlightweight as bflw
import pandas as pd
from pandas.io.json import json_normalize as jn
import os
from dotenv import load_dotenv, find_dotenv
import bz2

In [72]:
# Set view parameters
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Set figsize for charting
# plt.rcParams['figure.figsize'] = (16, 8)

# Get login creds
dotenv_path = find_dotenv()
load_dotenv(dotenv_path)

username = os.environ.get('BF_USERNAME')
password = os.environ.get('BF_PASSWORD')
app_delay_key = os.environ.get('API_DELAY_KEY')
certs = '/Users/user/Code/Football/betfair/certs'
dl_path = '/Users/user/Code/Football/betfair/data'

In [110]:
# Log in
trading = bflw.APIClient(username, password, app_key=app_delay_key, certs=certs)

trading.login()

<LoginResource>

In [111]:
file_list = trading.historic.get_file_list(
    'Soccer', 'Basic Plan', 1, 1, 2018, 31, 1, 2018,
    countries_collection=['GB', 'IE'], market_types_collection=['MATCH_ODDS'], file_type_collection=['M'])

for file in file_list:
    print(file)
    download = trading.historic.download_file(file_path=file)
    print(download)

/data/xds/historic/BASIC/28541000/1.138733960.bz2
1.138733960.bz2
/data/xds/historic/BASIC/28526000/1.138274884.bz2
1.138274884.bz2
/data/xds/historic/BASIC/28529000/1.138400456.bz2
1.138400456.bz2
/data/xds/historic/BASIC/28529000/1.138400517.bz2
1.138400517.bz2
/data/xds/historic/BASIC/28537200/1.138616157.bz2
1.138616157.bz2
/data/xds/historic/BASIC/28552300/1.139064343.bz2
1.139064343.bz2
/data/xds/historic/BASIC/28549300/1.138998047.bz2
1.138998047.bz2
/data/xds/historic/BASIC/28527400/1.138322132.bz2
1.138322132.bz2
/data/xds/historic/BASIC/28527400/1.138473119.bz2
1.138473119.bz2
/data/xds/historic/BASIC/28535500/1.138582885.bz2
1.138582885.bz2
/data/xds/historic/BASIC/28537500/1.138631135.bz2
1.138631135.bz2
/data/xds/historic/BASIC/28552700/1.139073296.bz2
1.139073296.bz2
/data/xds/historic/BASIC/28543800/1.138806673.bz2
1.138806673.bz2
/data/xds/historic/BASIC/28543800/1.138806736.bz2
1.138806736.bz2
/data/xds/historic/BASIC/28543900/1.138811285.bz2
1.138811285.bz2
/data/xds/

In [3]:
uncompressedData = bz2.BZ2File('1.138807922.bz2').read()

In [4]:
uncompressedData

b'{"op":"mcm","clk":"5879313651","pt":1516186922375,"mc":[{"id":"1.138807922","marketDefinition":{"bspMarket":false,"turnInPlayEnabled":true,"persistenceEnabled":true,"marketBaseRate":5.0,"eventId":"28543811","eventTypeId":"1","numberOfWinners":1,"bettingType":"ODDS","marketType":"MATCH_ODDS","marketTime":"2018-01-20T15:00:00.000Z","suspendTime":"2018-01-20T15:00:00.000Z","bspReconciled":false,"complete":true,"inPlay":false,"crossMatching":false,"runnersVoidable":false,"numberOfActiveRunners":3,"betDelay":0,"status":"OPEN","runners":[{"status":"ACTIVE","sortPriority":1,"id":18563,"name":"Peterborough"},{"status":"ACTIVE","sortPriority":2,"id":69819,"name":"Oldham"},{"status":"ACTIVE","sortPriority":3,"id":58805,"name":"The Draw"}],"regulators":["MR_ESP"],"countryCode":"GB","discountAllowed":true,"timezone":"Europe/London","openDate":"2018-01-20T15:00:00.000Z","version":2011829403,"name":"Match Odds","eventName":"Peterborough v Oldham"}}]}\n{"op":"mcm","clk":"5888133622","pt":1516313919

In [74]:
trading.historic.get_my_data()

[{'sport': 'Soccer',
  'plan': 'Basic Plan',
  'forDate': '2015-04-01T00:00:00',
  'purchaseItemId': 25482},
 {'sport': 'Soccer',
  'plan': 'Basic Plan',
  'forDate': '2015-05-01T00:00:00',
  'purchaseItemId': 25482},
 {'sport': 'Soccer',
  'plan': 'Basic Plan',
  'forDate': '2015-06-01T00:00:00',
  'purchaseItemId': 25482},
 {'sport': 'Soccer',
  'plan': 'Basic Plan',
  'forDate': '2015-07-01T00:00:00',
  'purchaseItemId': 25482},
 {'sport': 'Soccer',
  'plan': 'Basic Plan',
  'forDate': '2015-08-01T00:00:00',
  'purchaseItemId': 25482},
 {'sport': 'Soccer',
  'plan': 'Basic Plan',
  'forDate': '2015-09-01T00:00:00',
  'purchaseItemId': 25482},
 {'sport': 'Soccer',
  'plan': 'Basic Plan',
  'forDate': '2015-10-01T00:00:00',
  'purchaseItemId': 25482},
 {'sport': 'Soccer',
  'plan': 'Basic Plan',
  'forDate': '2015-11-01T00:00:00',
  'purchaseItemId': 25482},
 {'sport': 'Soccer',
  'plan': 'Basic Plan',
  'forDate': '2015-12-01T00:00:00',
  'purchaseItemId': 25482},
 {'sport': 'Soccer'

In [37]:
collection_options = trading.historic.get_collection_options('Soccer', 'Basic Plan', 1, 1, 2018, 31, 1, 2018)

In [46]:
jn(collection_options)

,countriesCollection,fileTypeCollection,marketTypesCollection
0,"[{'name': '', 'count': 16621}, {'name': 'AE', ...","[{'name': 'E', 'count': 4818}, {'name': 'M', '...","[{'name': '', 'count': 5067}, {'name': 'AH_ODD..."


In [48]:
jn(collection_options['marketTypesCollection'])

,count,name
0,5067,
1,9,AH_ODDS_MARKET
2,4370,ALT_TOTAL_GOALS
3,33,ANYTIME_SCORE
4,4397,ASIAN_HANDICAP
5,33,BOOKING_MATCH_BET
6,200,BOOKING_ODDS
7,4700,BOTH_TEAMS_TO_SCORE
8,1493,CLEAN_SHEET
9,131,CORNER_MATCH_BET


In [75]:
# Get advance basket data size
basket_size = trading.historic.get_data_size('Soccer', 'Basic Plan', 1, 1, 2018, 31, 1, 2018)
print(basket_size)

{'totalSizeMB': 250, 'fileCount': 135174}


In [89]:
file_list = trading.historic.get_file_list(
    'Soccer', 'Basic Plan', 1, 8, 2018, 31, 9, 2018,
    countries_collection=['GB', 'IE'], market_types_collection=['MATCH_ODDS'], file_type_collection=['M'])

for file in file_list:
    print(file)
    download = trading.historic.download_file(file_path=file)
    print(download)

/data/xds/historic/BASIC/28823244/1.146146447.bz2
1.146146447.bz2
/data/xds/historic/BASIC/28824255/1.146165371.bz2
1.146165371.bz2
/data/xds/historic/BASIC/28825432/1.146195410.bz2
1.146195410.bz2
/data/xds/historic/BASIC/28825433/1.146195501.bz2
1.146195501.bz2
/data/xds/historic/BASIC/28825431/1.146195319.bz2
1.146195319.bz2
/data/xds/historic/BASIC/28825531/1.146197947.bz2
1.146197947.bz2
/data/xds/historic/BASIC/28820718/1.146100775.bz2
1.146100775.bz2
/data/xds/historic/BASIC/28810510/1.145836522.bz2
1.145836522.bz2
/data/xds/historic/BASIC/28810511/1.145836646.bz2
1.145836646.bz2
/data/xds/historic/BASIC/28818304/1.146047545.bz2
1.146047545.bz2
/data/xds/historic/BASIC/28818319/1.146048797.bz2
1.146048797.bz2
/data/xds/historic/BASIC/28828447/1.146260382.bz2
1.146260382.bz2
/data/xds/historic/BASIC/28818827/1.146053637.bz2
1.146053637.bz2
/data/xds/historic/BASIC/28828454/1.146260565.bz2
1.146260565.bz2
/data/xds/historic/BASIC/28824472/1.146173927.bz2
1.146173927.bz2
/data/xds/

ConnectionError: ('Connection aborted.', OSError("(54, 'ECONNRESET')",))